### Health Check

In [1]:
[ordered]@{"$((Get-TimeZone).Id)" = Get-Date
    'Boot'      = (Get-CimInstance win32_operatingsystem).lastbootuptime.ToString('ddd M.d.yy H:m')
    'CpuUsage%' = (Get-CimInstance win32_processor | Measure-Object LoadPercentage -Average).Average
    'RamUsage%' = {$os=Get-CimInstance win32_operatingsystem; [Math]::Ceiling(($os.TotalVisibleMemorySize-$os.FreePhysicalMemory)*100/$os.TotalVisibleMemorySize)}.Invoke()
    'C:%Free'   = { $disk = Get-CimInstance Win32_LogicalDisk -Filter "DeviceID='C:'"; [Math]::Ceiling(($disk.FreeSpace / $disk.Size) * 100) }.Invoke()
            } | ft -HideTableHeaders -AutoSize


Central Standard Time 12/3/2023 2:53:11 PM
Boot                  Sun 12.3.23 11:9
CpuUsage%             79
RamUsage%             {48}
C:%Free               {55}



This shows which processes are consuming the most cpu, memory and network and have the most number of handles and threads currently on the system.

In [5]:
$Host.UI.RawUI.BufferSize = New-Object Management.Automation.Host.Size(130, $Host.UI.RawUI.BufferSize.Height)  #v2: One line equivalent

$objArr = Get-Process | Select-Object @{Name='Process'; Expression={$_.ProcessName+'►'+$_.Id}}, CPU, ID, WS, HandleCount, Threads
$Network = (Get-Counter '\Process(*)\IO Read Operations/sec').CounterSamples.Where({$_.InstanceName -ne "_Total" -and $_.InstanceName -ne "Idle"}) | sort cookedvalue -Descending | Select -Property instancename -First 10

# Get IO Read and Write Operations
$ioOps = Get-Counter '\Process(*)\IO Read Operations/sec', '\Process(*)\IO Write Operations/sec' |Select -ExpandProperty CounterSamples | Where-Object { $_.InstanceName -ne "_Total" -and $_.InstanceName -ne "Idle" }
$combinedIO = @{}
foreach ($op in $ioOps) {$combinedIO[$op.InstanceName] += $op.CookedValue}  # Combine and sum the IO operations for each process
$DiskIOPS = $combinedIO.GetEnumerator() | Sort-Object Value -Descending | Select-Object -First 10 -ExpandProperty Name

$sortCriteria = @('CPU', 'WS', 'HandleCount', 'Threads')   # Using a loop to sort for each property
$sortedResults = @{}
foreach ($criteria in $sortCriteria) {$sortedResults[$criteria] = $objArr | Sort-Object $criteria -Descending | Select-Object -First 10}

$harray = for ($j = 0; $j -lt 10; $j++) {  # Creating the custom object array
    [PSCustomObject][ordered]@{
        'CPU'     = $sortedResults['CPU'][$j].Process 
        'WS'      = $sortedResults['WS'][$j].Process 
        'Handles' = $sortedResults['HandleCount'][$j].Process 
        'Threads' = $sortedResults['Threads'][$j].Process
        'Network' = $Network[$j].InstanceName
        'DiskIOPS' = $DiskIOPS[$j]
    }
}
$harray | Format-Table -AutoSize


CPU                     WS                      Handles                 Threads                 Network           DiskIOPS
---                     --                      -------                 -------                 -------           --------
msedge►13512            Memory Compression►2632 OneDrive►11916          System►4                code              code
msedge►6124             dwm►772                 System►4                Memory Compression►2632 code              msedge
OneDrive►11916          msedge►7464             explorer►6532           explorer►6532           code              svchost
msedge►7464             WindowsTerminal►9304    msedge►7464             msedge►7464             dwm               powershell_ise
Grammarly.Desktop►12308 Code►5344               lsass►904               SearchHost►6964         msedge            dotnet
Code►5344               pwsh►9668               Grammarly.Desktop►12308 Code►11620              grammarly.desktop wslservice
Code►16052      